<img src='logo/dsl-logo.png' width="500" align="center" />

# HR Competition

## Random Forest Model for Kaggle

### Initializations

In [51]:
# Bibliotheken einbinden
import numpy as np
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import learning_curve
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

### Load Data

In [52]:
df = pd.read_pickle('exchange/hr_01_enriched_train.pkl')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
satisfactionLevel                  10000 non-null float64
yearsSinceEvaluation               10000 non-null float64
numberOfProjects                   10000 non-null int64
averageMonthlyHours                10000 non-null int64
yearsAtCompany                     10000 non-null int64
workAccident                       10000 non-null category
hasLeftCompany                     10000 non-null category
gotPromotion                       10000 non-null category
department                         10000 non-null category
salary                             10000 non-null category
projectsPerYear                    10000 non-null float64
hoursPerProject                    10000 non-null float64
satisfactionHours                  10000 non-null float64
workingHoursSinceLastEvaluation    10000 non-null float64
evaluationSatisfaction             10000 non-null float64
projectsPerWorkingHour 

In [53]:
# Datentyp von Category in Object umwandeln
for col in df.select_dtypes(['category']):
    print('transforming', col)
    df[col] = df[col].astype('str')

transforming workAccident
transforming hasLeftCompany
transforming gotPromotion
transforming department
transforming salary


In [54]:
df = pd.get_dummies(df.drop(['hasLeftCompany'], axis=1)).join(df[['hasLeftCompany']])
df.head()

,satisfactionLevel,yearsSinceEvaluation,numberOfProjects,averageMonthlyHours,yearsAtCompany,projectsPerYear,hoursPerProject,satisfactionHours,workingHoursSinceLastEvaluation,evaluationSatisfaction,...,department_management,department_marketing,department_product_mng,department_sales,department_support,department_technical,salary_high,salary_low,salary_medium,hasLeftCompany
0,0.65,0.96,5,226,2,2.500000,1084.8,146.90,2603.52,0.6240,...,0,1,0,0,0,0,0,0,1,0
1,0.88,0.80,3,166,2,1.500000,1328.0,146.08,1593.60,0.7040,...,0,0,0,0,0,0,0,1,0,0
2,0.69,0.98,3,214,2,1.500000,1712.0,147.66,2516.64,0.6762,...,0,0,0,1,0,0,0,1,0,0
3,0.41,0.47,2,154,3,0.666667,2772.0,63.14,868.56,0.1927,...,0,0,0,1,0,0,0,1,0,1
4,0.87,0.76,5,254,2,2.500000,1219.2,220.98,2316.48,0.6612,...,0,0,0,0,0,0,0,1,0,0


In [55]:
y_train = df['hasLeftCompany'].values
y_train

array(['0', '0', '0', ..., '0', '0', '1'], dtype=object)

In [56]:
X_train = df.drop(['hasLeftCompany'], axis=1).values
X_train

array([[ 0.65,  0.96,  5.  , ...,  0.  ,  0.  ,  1.  ],
       [ 0.88,  0.8 ,  3.  , ...,  0.  ,  1.  ,  0.  ],
       [ 0.69,  0.98,  3.  , ...,  0.  ,  1.  ,  0.  ],
       ..., 
       [ 0.83,  0.86,  4.  , ...,  0.  ,  1.  ,  0.  ],
       [ 0.74,  0.56,  4.  , ...,  0.  ,  1.  ,  0.  ],
       [ 0.11,  0.88,  7.  , ...,  0.  ,  0.  ,  1.  ]])

In [57]:
scaler = MinMaxScaler()

In [58]:
X_train_scaled = scaler.fit_transform(X_train)

In [59]:
df = pd.read_pickle('exchange/hr_01_enriched_test.pkl')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4999 entries, 0 to 4998
Data columns (total 17 columns):
id                                 4999 non-null int64
satisfactionLevel                  4999 non-null float64
yearsSinceEvaluation               4999 non-null float64
numberOfProjects                   4999 non-null int64
averageMonthlyHours                4999 non-null int64
yearsAtCompany                     4999 non-null int64
workAccident                       4999 non-null category
gotPromotion                       4999 non-null category
department                         4999 non-null category
salary                             4999 non-null category
projectsPerYear                    4999 non-null float64
hoursPerProject                    4999 non-null float64
satisfactionHours                  4999 non-null float64
workingHoursSinceLastEvaluation    4999 non-null float64
evaluationSatisfaction             4999 non-null float64
projectsPerWorkingHour             4999 no

In [60]:
# Datentyp von Category in Object umwandeln
for col in df.select_dtypes(['category']):
    print('transforming', col)
    df[col] = df[col].astype('str')

transforming workAccident
transforming gotPromotion
transforming department
transforming salary


In [61]:
df = pd.get_dummies(df.drop(['id'], axis=1)).join(df[['id']])
df.head()

,satisfactionLevel,yearsSinceEvaluation,numberOfProjects,averageMonthlyHours,yearsAtCompany,projectsPerYear,hoursPerProject,satisfactionHours,workingHoursSinceLastEvaluation,evaluationSatisfaction,...,department_management,department_marketing,department_product_mng,department_sales,department_support,department_technical,salary_high,salary_low,salary_medium,id
0,0.81,0.96,4,219,2,2.000000,1314.0,177.39,2522.88,0.7776,...,0,0,0,0,0,1,0,1,0,10000
1,0.86,0.84,4,246,6,0.666667,4428.0,211.56,2479.68,0.7224,...,0,0,0,0,0,0,0,1,0,10001
2,0.90,0.66,4,242,3,1.333333,2178.0,217.80,1916.64,0.5940,...,0,0,0,0,1,0,1,0,0,10002
3,0.37,0.54,2,131,3,0.666667,2358.0,48.47,848.88,0.1998,...,0,0,0,0,0,0,0,0,1,10003
4,0.52,0.96,3,271,3,1.000000,3252.0,140.92,3121.92,0.4992,...,0,0,0,0,0,1,0,0,1,10004


In [62]:
ids = df['id']
ids.head()

0    10000
1    10001
2    10002
3    10003
4    10004
Name: id, dtype: int64

In [63]:
X_test = df.drop(['id'], axis=1).values
X_test

array([[ 0.81,  0.96,  4.  , ...,  0.  ,  1.  ,  0.  ],
       [ 0.86,  0.84,  4.  , ...,  0.  ,  1.  ,  0.  ],
       [ 0.9 ,  0.66,  4.  , ...,  1.  ,  0.  ,  0.  ],
       ..., 
       [ 0.66,  0.73,  5.  , ...,  0.  ,  0.  ,  1.  ],
       [ 0.79,  1.  ,  4.  , ...,  0.  ,  1.  ,  0.  ],
       [ 0.98,  0.86,  2.  , ...,  0.  ,  1.  ,  0.  ]])

In [64]:
X_test_scaled = scaler.transform(X_test)

### Predict Kaggle Data

Nachdem bisher sechs unterschiedliche Modell sehr ähnliche Ergebnisse produziert hatten, wurde nun noch ein Versuch unternommen, die einzelnen Estimator durch einen Soft Voting Classifier zusammenspielen zu lassen. 

In [65]:
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

In [66]:
clf1 = RandomForestClassifier(criterion='gini', max_features=0.6, n_estimators=300)
clf2 = GradientBoostingClassifier(max_depth=9, n_estimators=1500, learning_rate=0.05)
clf3 = GradientBoostingClassifier(learning_rate=0.0125, n_estimators=1440,max_depth=11,min_samples_split=240, min_samples_leaf=1, subsample=0.8, random_state=10,max_features=8)
clf4 = GradientBoostingClassifier(learning_rate=0.0375, n_estimators=420,max_depth=11,min_samples_split=320, min_samples_leaf=2, subsample=0.8, random_state=10,max_features=8)
clf5 = RandomForestClassifier(max_features=0.3, n_estimators=200)
clf6 = GradientBoostingClassifier(learning_rate=0.5, n_estimators=200,max_depth=30,min_samples_split=32, min_samples_leaf=3, max_features=15, random_state=10)

In [67]:
eclf = VotingClassifier(estimators=[('clf1', clf1), ('clf2', clf2), ('clf3', clf3), ('clf4', clf4), ('clf5', clf5), ('clf6', clf6)], voting='soft')

In [68]:
clf1 = clf1.fit(X_train_scaled, y_train)
clf2 = clf2.fit(X_train_scaled, y_train)
clf3 = clf3.fit(X_train_scaled, y_train)
clf4 = clf4.fit(X_train_scaled, y_train)
clf5 = clf5.fit(X_train_scaled, y_train)
clf6 = clf6.fit(X_train_scaled, y_train)
eclf = eclf.fit(X_train_scaled, y_train)

In [69]:
scores = cross_val_score(eclf, X_train_scaled, y_train, cv=10, n_jobs=-1)
scores.mean()

0.98770048770048768

In [70]:
predictions = eclf.predict(X_test_scaled)
list(predictions);

In [71]:
list(ids);

In [72]:
df = pd.DataFrame(
    {'id': ids,
     'left': predictions
    })
df.head()

,id,left
0,10000,0
1,10001,1
2,10002,0
3,10003,1
4,10004,0


In [73]:
df.to_csv('kaggle/voting_ensemble.csv', index=False)

**Ergebnis in Kaggle:** 98,982%

## Summary

Das Ergebnis der Analyse der Personaldaten zeigt, dass es verschiedene Gruppen von Mitarbeitern gibt, die das Unternehmen verlassen. Es lässt sich sagen, dass im Besonderen anhand der Features `satisfaction`, `yearsAtCompany`, und `evaluation` auf die Kündigungswahrscheinlichkeit der Mitarbeiter geschlossen werden kann. 
Ein herausstechender Prädiktor ist das Zufriedenheitsniveau: unzufriedene Mitarbeiter verlassen sehr wahrscheinlich das Unternehmen. So konnte auch die Vermutung „Je geringer die Zufriedenheit, desto höher ist die Chance, dass das Unternehmen verlassen wird“ bestätigt werden.
Zudem sind Mitarbeiter, bei denen die Bewertung relativ lange her ist (mehr als 9 Monate), ebenfalls stark gefährdet, zu kündigen. 	
Ein weiterer wichtiger Faktor ist die Anzahl der Jahre, die der Mitarbeiter bereits beim Unternehmen ist. Diejenigen, die vier bis fünf Jahre bei der Organisation arbeiten, beeinflussen neben den bereits genannten Faktoren die Fluktuationsrate maßgeblich.
Nicht außer Acht gelassen werden sollte auch die Anzahl der Stunden, die ein Mitarbeiter im Monat bzw. in der Woche arbeitet. Hier konnten zwei Gruppen ausgemacht werden: die Unterforderten, die weniger als 150 Stunden im Monat bzw. weniger als 6 Stunden pro Tag arbeiten sowie die Überarbeiteten, die mehr als 250 Stunden im Monat bzw. mehr als 10 Stunden pro Tag mit Arbeit verbringen. 
Weiterhin zeigte sich, dass Mitarbeiter mit zwei, sechs oder sieben Projekten – sprich, mit relativ wenigen oder vielen Projekten, ebenfalls eher das Unternehmen verlassen. 



Um die Features in Zusammenhang zu bringen wurde überpüft, ob man sich Maßnahmen überlegen sollte, sodass „High Potentials“ weiterhin in der Firma gehalten werden können. Zunächst ist dafür analysiert worden, ob die sogenannten High Potentials die Firma vermehrt verlassen. 
Die High Potentials wurden wie folgt definiert: Mitarbeiter, die weniger als 230 Stunden pro Monat arbeiten, sodass sie nicht Burn Out gefährdet  sowie gleichzeitig ein Zufriedenheitslevel von 0,5 oder höher aufweisen und entweder mit mehr als 1,5 Projekten pro Jahr eine hohe Projektwiederholungsrate aufweist oder eine Beförderung erhalten haben. 
Die Geschäftsführung der Firma kann beruhigt werden, denn die „wertvollen Mitarbeiter“ kündigen deutlich seltener als Mitarbeiter, die nicht in die Kategorie „wertvoller Mitarbeiter“ eingeordnet werden. 


Die besten Modellergebnisse mit 99,13% konnten auf Kaggle mit den Ensemble-Verfahren Random Forest und Gradient Boost erzielt werden. 